# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 15 2022 12:09PM,250807,19,MSP215782,"Methapharm, Inc.",Released
1,Nov 15 2022 12:09PM,250807,19,MSP215795,"Methapharm, Inc.",Released
2,Nov 15 2022 12:09PM,250807,19,MSP215813,"Methapharm, Inc.",Released
3,Nov 15 2022 12:09PM,250807,19,MSP215814,"Methapharm, Inc.",Released
4,Nov 15 2022 12:09PM,250807,19,MSP215840,"Methapharm, Inc.",Released
5,Nov 15 2022 12:09PM,250807,19,MSP215843,"Methapharm, Inc.",Released
6,Nov 15 2022 12:09PM,250807,19,MSP215845,"Methapharm, Inc.",Released
7,Nov 15 2022 12:09PM,250807,19,MSP215855,"Methapharm, Inc.",Released
8,Nov 15 2022 12:09PM,250807,19,MSP215862,"Methapharm, Inc.",Released
9,Nov 15 2022 12:09PM,250807,19,MSP215871,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,250803,Released,4
28,250804,Released,1
29,250805,Released,1
30,250806,Released,1
31,250807,Released,10


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250803,NaN,NaN,4.0
250804,NaN,NaN,1.0
250805,NaN,NaN,1.0
250806,NaN,NaN,1.0
250807,NaN,NaN,10.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250670,0.0,0.0,48.0
250703,23.0,1.0,0.0
250754,0.0,5.0,0.0
250755,0.0,4.0,25.0
250757,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250670,0,0,48
250703,23,1,0
250754,0,5,0
250755,0,4,25
250757,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250670,0,0,48
1,250703,23,1,0
2,250754,0,5,0
3,250755,0,4,25
4,250757,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250670,,,48
1,250703,23,1,
2,250754,,5,
3,250755,,4,25
4,250757,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 15 2022 12:09PM,250807,19,"Methapharm, Inc."
10,Nov 15 2022 12:08PM,250806,10,Emerginnova
11,Nov 15 2022 12:07PM,250805,15,Lonza
12,Nov 15 2022 12:07PM,250802,20,"Exact-Rx, Inc."
24,Nov 15 2022 12:05PM,250804,19,"GUSA Granules USA, Inc."
25,Nov 15 2022 12:05PM,250803,10,MedStone Pharma LLC
29,Nov 15 2022 12:00PM,250801,19,"GUSA Granules USA, Inc."
30,Nov 15 2022 11:43AM,250800,10,ISDIN Corporation
51,Nov 15 2022 11:38AM,250787,10,ISDIN Corporation
69,Nov 15 2022 11:37AM,250799,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 15 2022 12:09PM,250807,19,"Methapharm, Inc.",,,10
1,Nov 15 2022 12:08PM,250806,10,Emerginnova,,,1
2,Nov 15 2022 12:07PM,250805,15,Lonza,,,1
3,Nov 15 2022 12:07PM,250802,20,"Exact-Rx, Inc.",,,12
4,Nov 15 2022 12:05PM,250804,19,"GUSA Granules USA, Inc.",,,1
5,Nov 15 2022 12:05PM,250803,10,MedStone Pharma LLC,,,4
6,Nov 15 2022 12:00PM,250801,19,"GUSA Granules USA, Inc.",,,1
7,Nov 15 2022 11:43AM,250800,10,ISDIN Corporation,,,21
8,Nov 15 2022 11:38AM,250787,10,ISDIN Corporation,,,18
9,Nov 15 2022 11:37AM,250799,10,ISDIN Corporation,,,8


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 12:09PM,250807,19,"Methapharm, Inc.",10,,
1,Nov 15 2022 12:08PM,250806,10,Emerginnova,1,,
2,Nov 15 2022 12:07PM,250805,15,Lonza,1,,
3,Nov 15 2022 12:07PM,250802,20,"Exact-Rx, Inc.",12,,
4,Nov 15 2022 12:05PM,250804,19,"GUSA Granules USA, Inc.",1,,
5,Nov 15 2022 12:05PM,250803,10,MedStone Pharma LLC,4,,
6,Nov 15 2022 12:00PM,250801,19,"GUSA Granules USA, Inc.",1,,
7,Nov 15 2022 11:43AM,250800,10,ISDIN Corporation,21,,
8,Nov 15 2022 11:38AM,250787,10,ISDIN Corporation,18,,
9,Nov 15 2022 11:37AM,250799,10,ISDIN Corporation,8,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 12:09PM,250807,19,"Methapharm, Inc.",10,,
1,Nov 15 2022 12:08PM,250806,10,Emerginnova,1,,
2,Nov 15 2022 12:07PM,250805,15,Lonza,1,,
3,Nov 15 2022 12:07PM,250802,20,"Exact-Rx, Inc.",12,,
4,Nov 15 2022 12:05PM,250804,19,"GUSA Granules USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 12:09PM,250807,19,"Methapharm, Inc.",10.0,NaN,NaN
1,Nov 15 2022 12:08PM,250806,10,Emerginnova,1.0,NaN,NaN
2,Nov 15 2022 12:07PM,250805,15,Lonza,1.0,NaN,NaN
3,Nov 15 2022 12:07PM,250802,20,"Exact-Rx, Inc.",12.0,NaN,NaN
4,Nov 15 2022 12:05PM,250804,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 12:09PM,250807,19,"Methapharm, Inc.",10.0,0.0,0.0
1,Nov 15 2022 12:08PM,250806,10,Emerginnova,1.0,0.0,0.0
2,Nov 15 2022 12:07PM,250805,15,Lonza,1.0,0.0,0.0
3,Nov 15 2022 12:07PM,250802,20,"Exact-Rx, Inc.",12.0,0.0,0.0
4,Nov 15 2022 12:05PM,250804,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2758649,80.0,20.0,5.0
12,501573,2.0,0.0,0.0
15,752243,90.0,18.0,0.0
16,250766,1.0,0.0,0.0
19,1253903,13.0,1.0,23.0
20,501573,17.0,0.0,4.0
21,752344,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2758649,80.0,20.0,5.0
1,12,501573,2.0,0.0,0.0
2,15,752243,90.0,18.0,0.0
3,16,250766,1.0,0.0,0.0
4,19,1253903,13.0,1.0,23.0
5,20,501573,17.0,0.0,4.0
6,21,752344,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,80.0,20.0,5.0
1,12,2.0,0.0,0.0
2,15,90.0,18.0,0.0
3,16,1.0,0.0,0.0
4,19,13.0,1.0,23.0
5,20,17.0,0.0,4.0
6,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,80.0
1,12,Released,2.0
2,15,Released,90.0
3,16,Released,1.0
4,19,Released,13.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,5.0,0.0,0.0,0.0,23.0,4.0,0.0
Executing,20.0,0.0,18.0,0.0,1.0,0.0,0.0
Released,80.0,2.0,90.0,1.0,13.0,17.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,5.0,0.0,0.0,0.0,23.0,4.0,0.0
1,Executing,20.0,0.0,18.0,0.0,1.0,0.0,0.0
2,Released,80.0,2.0,90.0,1.0,13.0,17.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,5.0,0.0,0.0,0.0,23.0,4.0,0.0
1,Executing,20.0,0.0,18.0,0.0,1.0,0.0,0.0
2,Released,80.0,2.0,90.0,1.0,13.0,17.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()